In [36]:
from __future__ import print_function
import pandas as pd
import numpy as np

In [25]:
def rev_comp(strSeq):
	dicComp = {'A':'T','T':'A','G':'C','C':'G','N':'N'}
	strCseq = ''
	for i in strSeq:
		try:
			strCseq += dicComp[i.upper()]
		except KeyError:
			strCseq += 'N'
	# End of for i
	return(strCseq[::-1])

In [26]:
file_fa = '/ref/analysis/Cre/ref/Creinhardtii_281_v5.0.fa'

bulk = open(file_fa).read()
bulk = bulk.split('>')
dicHD2seq = {}
for each_bulk in bulk:
    if each_bulk.strip() == '':
        continue
    genename = each_bulk.split('\n')[0]
    seq      = ''.join(each_bulk.split('\n')[1:])
    dicHD2seq[genename] = seq

In [27]:
file_gff = '/ref/analysis/stringtie.addcds/my_csv.csv.addgene.gff3.sort.gff3'

df_gff = pd.read_csv(file_gff,sep='\t',header=None,comment='#')

mask        = (df_gff[2] == 'gene')
df_gff_gene = df_gff[mask]      

df_gff_gene['genename'] = df_gff[8].apply(lambda x:x.replace('ID=',''))

mask = (df_gff[2] == 'CDS')
df_gff_cds = df_gff[mask]      

df_gff_cds['genename'] = df_gff_cds[8].apply(lambda x : x.replace('Parent=',''))

df_gff_cds_index = df_gff_cds.set_index('genename')

df_gff_cds_index = df_gff_cds_index.sort([3], ascending=[1])

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:17: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [28]:
df_gff.head()

,0,1,2,3,4,5,6,7,8
0,chromosome_1,ST/ALL/KYJ,gene,9027,10985,.,+,.,ID=gene1
1,chromosome_1,ST/AG/CRE,mRNA,9027,10985,.,+,.,ID=mRNA1;Parent=gene1;Name=STRG.27.2
2,chromosome_1,ST/AG/CRE,five_prime_UTR,9027,9229,.,+,.,Parent=mRNA1;Name=STRG.27.2
3,chromosome_1,ST/AG/CRE,CDS,9230,9472,.,+,0,Parent=mRNA1;Name=STRG.27.2
4,chromosome_1,ST/AG/CRE,three_prime_UTR,9473,10985,.,+,.,Parent=mRNA1;Name=STRG.27.2


In [29]:
i = 0
dic = {'transcriptname' : [],
       'strand' : [],
       'CDSloc' : [],
       'CDSseq' : []}
for genename in set(df_gff_cds_index.index):
    
    dic['transcriptname'].append(genename)
    
    edf = df_gff_cds_index.loc[genename]
    if str(type(edf)) == "<class 'pandas.core.frame.DataFrame'>":
        CDS_list = zip(list(edf[3]),list(edf[4]))
        chromosome = edf[0][0]
        strand     = edf[6][0]
    else:
        CDS_list = zip([edf[3]],[edf[4]])
        chromosome = edf[0]
        strand     = edf[6]
    
    dic['strand'].append(strand)
    cdsseq = ''
    for l,r in CDS_list:
        #print l,r
        cdsseq += dicHD2seq[chromosome][l-1:r]
    if strand == '+':
        pass
    else: 
        cdsseq = rev_comp(cdsseq)
    dic['CDSloc'].append(CDS_list)
    dic['CDSseq'].append(cdsseq)
    #print genename,chromosome,strand,CDS_list,cdsseq
    #if i == 10:
    #    break
    #i += 1

In [30]:
df_cds = pd.DataFrame(dic)
df_cds['transcript'] = df_cds['transcriptname'].apply(lambda x : x.split('.')[1])
df_cds['gene'] = df_cds['transcriptname'].apply(lambda x : x.split('.')[0])

In [31]:
df_cds_index = df_cds.set_index(['gene','transcript'])

In [32]:
df_cds_index.to_pickle(file_gff+'.cdsseq.pk')

In [33]:
df_cds_index.loc['mRNA6917;Name=STRG']['CDSseq'].values

array([ 'ATGATGCACTGCCTGCGTGGCCTGCGCGGGCTTACGGAGGGCCTTCCGTCCACGGTTGCAGCCCCGCTGCAGCACCTGGCGCCGGGCTTCTTTCGGTCACTGGTGGCGGATTCACCAGAGGCGGAGGCATGGGAGCAACATACAGCGAACAGCTGGCCTTTGTCGCCTCAGGACATGCATGTGAAGGAGCGGGGGCGGGGTGAGGAGGCCTCCACTTCACCACGTTGCAGGGGCGCGGCGTCGGCAGTGCGCGTACAGGGTGCCGCTCTGGGACGGCGAAGGGTGTTGCGGCTGGCTGGCTCTGACACCTTCGTTTCCAACGTGGCGACGCTGCTTTTAACACTTCAGCAGCAGCACGCGCCAGGGGATTCGCAGGCTGGGAGGGCCACAGTAGCTCGTGGATGCAGCCTGGCGTGGCGTGAAGCGGTGGGGGACCCACGCTTCCTGCAGCTCTTCCCGCGCCGAATCTCACCGGACACGAAGGACATCCTGCAGCGGCTGCAGGGCGGTGCGCGGAGCAGTGAACCAGGTGGTGCAGGCGGCAAGGCACGGGACATCGCTTGGACGCGGGAGGACGCGGCCCGGCACTCAGCGATGCTGCGGCTCATAGACCAGCTGCAGGAGGTCCAGCGCCAACTGCAGGAGGCTATGCCTCAGGACGAGGACTCGGACAGCGACGAGGACGGCACCGACAGCAGCTATGTAACGGTGGCTGCCTTCACGGTTGAAGACGGTTCCTTCATGTCGCAGCCCCGGGACCGTACGCGGTCAGCAAGCGGCAGCGCACGTGGCATGGACGGCGACATGGACGCAGCAGACGGCGGGCTCGGGGAGCAGGCGCCCAGCTCGTGGCCCTGTCCGCTGGGCCGGCGCCGGAGAGCGCCAGGCGGCCGGCTGCTGCGGGCCCCGGTGCCGCCTTGGGACGACACGCAGCCATTCCAGACGCACGTGGGCACAGTGTATCATTTTTGGCGGCTGGCTCCGTATGGCT

In [34]:
df_cds_index.head()

,,CDSloc,CDSseq,strand,transcriptname
gene,transcript,,,,
mRNA22572;Name=STRG,8216,"[(896879, 897025), (897493, 897604), (897878, ...",ATGGCAGGTTCCAAGAAGAAAAAGTCTGAGCCTTCGGGCCCAGACC...,-,mRNA22572;Name=STRG.8216.5
mRNA1659;Name=STRG,1093,"[(6704291, 6704576), (6704815, 6704901), (6705...",ATGGCGTCACTGCGTGCTCCCTGTGCCGCGAGCACATTGCCGACTC...,-,mRNA1659;Name=STRG.1093.3
mRNA18427;Name=STRG,4837,"[(226219, 226497), (226818, 227009), (227363, ...",ATGGTCCTTGTCGCGACATTAGGGTACTCGCAGCATGCCAAGTCGT...,-,mRNA18427;Name=STRG.4837.4
mRNA19204;Name=STRG,5527,"[(3670160, 3670182), (3671670, 3671725), (3672...",ATGTCGGTAGAGGTCGGCATAAAATGGAGCGGCAAGGAGTTTGTCA...,-,mRNA19204;Name=STRG.5527.3|m.265767
mRNA21255;Name=STRG,7186,"[(4429909, 4429973), (4430411, 4430519), (4431...",ATGGCGGCGCGGTCCCCGGTGGCTCAGTTCGAGCCTCAGGTCCGCA...,-,mRNA21255;Name=STRG.7186.1


In [39]:
with open('cds.fa','w') as f:
    for ix in df_cds_index.index:
        sub_df = df_cds_index.loc[ix]
        header = sub_df['transcriptname'].split(';')[1].replace('Name=','')
        seq    = sub_df['CDSseq']
        print('>'+header,file=f)
        print(seq,file=f)
